In [4]:
from thermo import ChemicalConstantsPackage
from thermo.unifac import UNIFAC, DOUFSG, DOUFIP2016
from chemicals import search_chemical
import numpy as np
import pandas as pd
from IPython.display import clear_output

In [6]:
with pd.ExcelFile('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/TestingData_Final.xlsx') as f:
    df = pd.read_excel(f)

In [8]:
y = []
IUPAC_mix = []
all_chem_mix = []
for i in range(len(df)):
    print(f'Datapoint {i+1} out of {len(df)}')
    c1 = df['Component 1'][i]
    c2 = df['Component 2'][i]
    IUPAC_mix += [[c1, c2]]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix[i])
    all_chem_mix += [constants.UNIFAC_Dortmund_groups]
    x = df['Composition component 1 [mol/mol]'][i]
    xs = [x, 1-x]
    T = df['Temperature [K]'][i]
    GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix[i], version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
    y += [GE.HE()]
    clear_output(wait=True)

Datapoint 4890 out of 4890


In [9]:
y

[-339.1989822840529,
 -339.1012198507226,
 -338.02746680379664,
 -335.82103029684777,
 -328.29148648144565,
 -603.2603014535985,
 -603.440081800681,
 -602.3933801656666,
 -599.292013113322,
 -587.3850626339025,
 -792.3355508707834,
 -793.0472234851147,
 -792.8391554386395,
 -789.8866482123519,
 -776.2929015033922,
 -906.4195734170164,
 -907.7906155328637,
 -908.9263670897104,
 -906.878863280596,
 -893.7826586384788,
 -945.36878256851,
 -947.3932474701361,
 -950.0524794348539,
 -949.3560298415265,
 -938.3839808507814,
 -908.915807948096,
 -911.4471757797488,
 -915.4629974259111,
 -916.2285882692767,
 -908.3911221686094,
 -796.6812526792712,
 -799.4247050624452,
 -804.2607640731306,
 -806.2367561727233,
 -801.8621938661325,
 -608.1831585959709,
 -610.6872858560898,
 -615.4129528963313,
 -617.9548241951198,
 -616.6154133876995,
 -342.8446347283913,
 -344.4925747656116,
 -347.7561722331278,
 -349.79341676098704,
 -350.22259181360755,
 -338.02746680379664,
 -335.82103029684777,
 -328.291486

In [ ]:
data = {'Component 1': np.array(IUPAC_mix)[:,0],
        'Component 2': np.array(IUPAC_mix)[:,1],
        'UNIFAC_DMD [J/mol]': y}
data = pd.DataFrame(data)
data.to_excel('Thermo_UNIFAC_DMD.xlsx', index=False, header=True, sheet_name='UNIFAC_DMD')

In [10]:
copy_df = df.copy()
copy_df['UNIFAC_DMD [J/mol]'] = y

In [12]:
copy_df.to_excel('/home/garren/Documents/MEng/Code/Latest_results/HPC Files/TestingData_Final.xlsx', index=False, header=True)

In [2]:
subset_df, subset_Indices, subset_Indices_T, Info_Indices, init_indices, init_indices_T = subsets(np.array(['all'])).get_subset_df()

In [3]:
N = len(Info_Indices['Component names']['Index'])
Idx_known = subset_df.iloc[subset_Indices_T[:,0],7:9].to_numpy()
Idx_unknown = np.array([[i, j] for i in range(N) for j in range(i+1,N)])
idx = np.sum(np.char.add(Idx_unknown[:,0].astype(str), Idx_unknown[:,1].astype(str))[:,np.newaxis] ==
             np.char.add(Idx_known[:,0].astype(str), Idx_known[:,1].astype(str))[np.newaxis,:], axis=1) == 0
Idx_unknown = Idx_unknown[idx,:]

In [4]:
T2 = np.array([288.15, 298.15, 308.15])
x2 = np.linspace(0,1, 21)[1:-1]

In [6]:
dic = {'Component 1': [], 'Component 2': [],
        'Composition component 1 [mol/mol]': [], 'Temperature [K]': [], 
        'UNIFAC_DMD [J/mol]': []}
        
for i in range(Idx_unknown.shape[0]):
    print(f'Datapoint {i+1} out of {Idx_unknown.shape[0]}')
    c1 = Info_Indices['Component names']['IUPAC'][Idx_unknown[i,0]]
    c2 = Info_Indices['Component names']['IUPAC'][Idx_unknown[i,1]]
    IUPAC_mix = [c1, c2]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix)
    all_chem_mix = constants.UNIFAC_Dortmund_groups
    for j in range(len(T2)):
        for k in range(len(x2)):
            dic['Composition component 1 [mol/mol]'] += [x2[k]]
            dic['Temperature [K]'] += [T2[j]]
            dic['Component 1'] += [c1]
            dic['Component 2'] += [c2]
            xs = [x2[k], 1-x2[k]]
            GE = UNIFAC.from_subgroups(T=T2[j], xs=xs, chemgroups=all_chem_mix, version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
            dic['UNIFAC_DMD [J/mol]'] += [GE.HE()]
    clear_output(wait=True)

Datapoint 3875 out of 3875


In [17]:
pd.DataFrame(dic).to_excel('Thermo_UNIFAC_DMD_unknown.xlsx', index=False, header=True, sheet_name='UNIFAC_DMD_unknown')

In [20]:
with pd.ExcelFile('/home/garren/Documents/MEng/Code/Latest_results/Alkane_Alcohol_Subset_of_Testing_Data_and_UNIFAC_obtained_from_Aspen.xlsx') as f:
    df = pd.read_excel(f, sheet_name='TestingData')

In [22]:
y = []

for i in range(len(df)):
    print(f'Datapoint {i+1} out of {len(df)}')
    c1 = df['Component 1'][i]
    c2 = df['Component 2'][i]
    IUPAC_mix = [c1, c2]
    constants,_ = ChemicalConstantsPackage.from_IDs(IUPAC_mix)
    all_chem_mix = constants.UNIFAC_Dortmund_groups
    x = df['Composition component 1 [mol/mol]'][i]
    xs = [x, 1-x]
    T = df['Temperature [K]'][i]
    GE = UNIFAC.from_subgroups(T=T, xs=xs, chemgroups=all_chem_mix, version=1, interaction_data=DOUFIP2016, subgroups=DOUFSG)
    y += [GE.HE()]
    clear_output(wait=True)


Datapoint 123 out of 123


In [25]:
pd.DataFrame({'UNIFAC_DMD [J/mol]': y}).to_excel('/home/garren/Documents/MEng/Code/Latest_results/Thermo_UNIFAC_DMD_TestingData.xlsx', index=False, header=True, sheet_name='UNIFAC_DMD_TestingData')